In [2]:
import nibabel as nib
import numpy as np
from os.path import join as opj
from copy import deepcopy
import numpy as np
import pandas as pd

In [3]:

# Save image routine
def save_image_as_nifti(header, affine, image_to_save, save_as, 
                        dtype_set=np.int16):
    """
    Function to save a given file as a nifti-image
    :param string original_image: name and directory of the original image
    :param ndarray image_to_save: image with dimensions (height, width, depth) to be saved in nifti format
    :param save_as: name and directory where the nifti should be stored
    :return: void
    """
    print("Save image as: ", save_as)
    # Generate new nifti file
    if save_as.endswith("mgz"):
        mgh_img = nib.MGHImage(image_to_save, affine, header)
    elif any(save_as.endswith(file_ext) for file_ext in ["nii", "nii.gz"]):
        mgh_img = nib.nifti1.Nifti1Pair(image_to_save, affine, header)

    if dtype_set is not None:
        mgh_img.set_data_dtype(np.dtype(dtype_set))  # not uint8 if aparc!!! (only goes till 255)

    if any(save_as.endswith(file_ext) for file_ext in ["mgz", "nii"]):
        nib.save(mgh_img, save_as)
    elif save_as.endswith("nii.gz"):
        # For correct outputs, nii.gz files should be saved using the nifti1 sub-module's save():
        nib.nifti1.save(mgh_img, save_as)


In [17]:
# MCRIB mapping
fdir ="/groups/ag-reuter/projects/datasets/MCRIB/VINN"

In [71]:
# Left Hippocampus: 1, Right Hippocampus: 2
# Left Amygdala: 3, Right Amygdala: 4
# Left Ventricle 5, Right Ventricle: 6
# Left GM: 7, Right GM: 8
# Left WM: 9, Right WM: 10

REDUCEDMAPPING_MCRIB_GT = {1: 17, 2: 53, 3: 18, 4: 54, 5: 4, 6: 43,
                           7: [i for i in range(1000, 1036)],
                           8: [i for i in range(2000, 2036)],
                           9: 2, 10: 41
                          }

REDUCEDMAPPING_MCRIB_Pred = {1: 1, 2:2, 3:3, 4:4, 5:49, 6:50,
                             7: [5, 7, 9, 11, 13, 15, 21, 23, 25, 27, 29, 31, 33, 35, 37, 39, 88],
                             8: [6, 8, 10, 12, 14, 16, 20, 22, 24, 26, 28, 30, 32, 34, 36, 38, 85],
                             9: [52, 54, 56, 58, 60, 62, 64, 66, 68, 70, 72, 74, 76, 78, 80, 82],
                             10: [51, 53, 55, 57, 59, 61, 63, 65, 67, 69, 71, 73, 75, 77, 79, 81]
                            }


# CSF (1) = Left-/Right-Lateral-Ventricle, CSF
# GM (2) = Hippocampus, Amygdala, GM Parts
# Exclude (4): Brainstem, intra-cranial BG, Cerebellum
# WM (3): CC, WM parts + Thalamus, 

REDUCEDMAPPPING_iBEAT_Pred = {1: [4, 43],
                              2: [i for i in range(1000, 2036)] + [17, 53, 18, 54],
                              3: [2, 41]}

call_order = {"dhcp_mcrib": [7, 8, 9, 10, 1, 2, 3, 4, 5, 6],
              "mcrib_ibeat": [3, 1, 2]}

def reduce_mcrib(ddir="/groups/ag-reuter/projects/datasets/MCRIB/VINN",
                 sbj="P01", mapping_gt=REDUCEDMAPPING_MCRIB_GT,
                 mapping_pred=REDUCEDMAPPING_MCRIB_Pred):
    # sbj_dir
    sbj_dir = opj(ddir, sbj, "mri")
    
    # load image gt
    gt = nib.load(opj(sbj_dir, "aparc.DKTatlas+aseg.mgz"))
    gt_d = np.asanyarray(gt.dataobj)
    gt_new = np.zeros(gt_d.shape)
    for i in [5, 9, 1, 2, 3, 4, 6, 10, 7, 8]: 
        mask = np.in1d(gt_d, mapping_gt[i]).reshape(gt_d.shape)
        # Reduce ground truth to labels we want
        gt_new[mask] = i
    
    save_image_as_nifti(image_to_save=gt_new, affine=gt.affine, 
                        header=gt.header, 
                        save_as=opj(sbj_dir, "aseg_mapped.mgz"))
    
    # load prediction (full)
    all_preds = glob.glob(sbj_dir + "/aseg.TestingSet_FastSurferVINN_T2_05_T2_LatentAug*_full.mgz")
    for pred in all_preds:
        pred_new = dhcp_mcrib(pred, mapping_pred, [7, 8, 9, 10, 1, 2, 3, 4, 5, 6])
    return gt_new, pred_new

def ibeat_mask(sbj):
    ibeat = nib.load(os.path.join(sbj, "mri/aseg.ibeat_T2_05_T2.mgz"))
    ibeat_d = np.asanyarray(ibeat.dataobj)
    
    mcrib = nib.load(os.path.join(sbj, "mri/aparc.DKTatlas+aseg.mgz"))
    mcrib_d = np.asanyarray(mcrib.dataobj)
    mask = np.in1d(mcrib_d, [9, 11, 12, 13, 17, 18, 26, 28, 48, 50, 51, 52, 53, 54, 58, 60]).reshape(mcrib_d.shape)
    ibeat_d[mask] = 0
    save_image_as_nifti(image_to_save=ibeat_d, affine=ibeat.affine, 
                        header=ibeat.header, 
                        save_as=opj(sbj, "mri/aseg.ibeat_T2_05_T2.mgz"))
    return ibeat_d
    
def dhcp_mcrib(pred, mapping_pred, id_list):
    pred_full = nib.load(pred)
    pred_d = np.asanyarray(pred_full.dataobj)
    pred_new = np.zeros(pred_d.shape)          
    # Perform the mapping
    for i in id_list:
        mask = np.in1d(pred_d, mapping_pred[i]).reshape(pred_d.shape)

        # Reduce ground truth to labels we want
        pred_new[mask] = i
    sbj= os.path.basename(os.path.dirname(os.path.dirname(pred)))
    if sbj in ["P02", "P03", "P04", "P08", "P09"] and os.path.basename(pred) == "aseg.nnUNet3D_T2_05_T2_full.mgz":
        # increase by one
        pred_new[pred_new != 0] += 1
        pred_new[pred_new == 3] = 1
        pred_new[pred_new == 5] = 3
        pred_new[pred_new == 7] = 5
        pred_new[pred_new == 9] = 7
        pred_new[pred_new == 11] = 9
        
        save_image_as_nifti(image_to_save=pred_new, 
                            affine=pred_full.affine, 
                            header=pred_full.header,
                   save_as=opj(pred[:-4] + "_mapped.mgz"))
    return pred_new
  

In [72]:
import glob
import os
fdir = "/groups/ag-reuter/projects/datasets/MCRIB/VINN"
mcrib = glob.glob(fdir + "/P*")
for sub in mcrib:
    #sbj = sub.split("/")[-1]
    try:
        output = ibeat_mask(sub)
        #output = dhcp_mcrib(sub, mapping_pred=REDUCEDMAPPING_MCRIB_Pred, id_list=call_order["dhcp_mcrib"])#reduce_mcrib(sbj=sbj)
    except FileNotFoundError as e:
        print(f"Subject {sbj} failed: Error {e}")

Save image as:  /groups/ag-reuter/projects/datasets/MCRIB/VINN/P01/mri/aseg.ibeat_T2_05_T2.mgz
Subject sub-140 failed: Error No such file or no access: '/groups/ag-reuter/projects/datasets/MCRIB/VINN/P01_1/mri/aseg.ibeat_T2_05_T2.mgz'
Save image as:  /groups/ag-reuter/projects/datasets/MCRIB/VINN/P02/mri/aseg.ibeat_T2_05_T2.mgz
Save image as:  /groups/ag-reuter/projects/datasets/MCRIB/VINN/P03/mri/aseg.ibeat_T2_05_T2.mgz
Save image as:  /groups/ag-reuter/projects/datasets/MCRIB/VINN/P04/mri/aseg.ibeat_T2_05_T2.mgz
Save image as:  /groups/ag-reuter/projects/datasets/MCRIB/VINN/P05/mri/aseg.ibeat_T2_05_T2.mgz
Save image as:  /groups/ag-reuter/projects/datasets/MCRIB/VINN/P06/mri/aseg.ibeat_T2_05_T2.mgz
Save image as:  /groups/ag-reuter/projects/datasets/MCRIB/VINN/P07/mri/aseg.ibeat_T2_05_T2.mgz
Save image as:  /groups/ag-reuter/projects/datasets/MCRIB/VINN/P08/mri/aseg.ibeat_T2_05_T2.mgz
Save image as:  /groups/ag-reuter/projects/datasets/MCRIB/VINN/P09/mri/aseg.ibeat_T2_05_T2.mgz
Save 

In [3]:
# FETA mapping
fdir = "/projects/datasets/FETA/FastSurfer"
# load image
feta_test_gt = nib.load(fdir + "/sub-001/dseg_orig.nii.gz")
gt_d = np.asanyarray(feta_test_gt.dataobj)

# load prediction (full)
feta_test_pred = nib.load(fdir + "/sub-001/aseg.TestingSet_FastSurferVINN_T2_LatentAug_full.mgz")
pred_d = np.asanyarray(feta_test_pred.dataobj)

In [4]:
# iBEAT mapping
base = "/projects/"
idir = base + "datasets/dHCP/iBEAT/processing"
fdir = base + "datasets/dHCP/Data"
# load image
feta_test_gt = nib.load(fdir + "/sub-CC00600XX06_ses-175800/sub-CC00600XX06_ses-175800_desc-drawem88_dseg_min.nii.gz")
gt_d = np.asanyarray(feta_test_gt.dataobj)

# load prediction (full)
feta_test_pred = nib.load(idir + "/sub-CC00600XX06_ses-175800/T2-skullstripped-tissue.nii.gz")
pred_d = np.asanyarray(feta_test_pred.dataobj)

# load training samples
training=base + "master-theses/henschell/FastInfantSurfer/data/dataset_split_large_testing_t1t2_meta.tsv"

In [6]:
# Get unique labels for ground truth
labels_pred = np.unique(pred_d)
mapping = {}
for lab in labels_pred:
    # Get labels intersecting with gt from pred
    mask = (pred_d == lab)
    mapping[lab] = np.unique(gt_d[mask], return_counts=True)

In [7]:
mapping

{0: (array([ 0,  6,  8,  9, 10, 13, 14, 16, 17, 18, 19, 22, 23, 24, 26, 27, 28,
         29, 36, 37, 38, 39, 65, 66, 83, 84], dtype=int16),
  array([21469530,        2,        2,        3,       15,       46,
               20,        8,       98,      119,      467,     4414,
             7028,        3,       35,       39,        4,       18,
              778,      427,        7,       37,     1260,     2600,
            48617,   229522])),
 1: (array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
         17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
         34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 48, 49, 50, 51,
         52, 53, 54, 55, 56, 58, 59, 60, 61, 62, 64, 65, 66, 67, 68, 69, 70,
         71, 72, 73, 74, 76, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88],
        dtype=int16),
  array([  1090,    551,    665,     67,    192,    597,   1590,    802,
           1788,    697,   1462,   1142,   1623,   3283,   3075,    686,

In [37]:
REDUCEMAPPING_FETA = {1: [83], # CSF  = CSF 
                      2: [i for i in range(20, 40)] + [i for i in range(5, 17)], # GM = All GM labels
                      3: [48, 1, 2, 3, 4] + [i for i in range(51, 83)], # WM = All WM labels, CC, Hippocampus?, Amygdala?
                      4: [50, 49], # Ventricles
                      5: [17, 18], # Cerebellum = Cerebellum
                      6: [40, 41, 42, 43, 44, 45, 46, 47, 86, 87], # Thalamus, Lentiform Nucleus, Caudate Nucleus (part of subthalamic nucleus)
                      7: [19], # Brainstem = Brainstem + Part of IntraCranial BG, Subthalamic Nucleus, 
                      0: [84, 88, 85]} # exclude Intracranial BG, Subthalamic Nucleus?

REDUCEMAPPING_FETA_IBEAT = {1: [1, 4], # CSF = CSF, Lateral Ventricles
                            2: [2, 6], # GM = GM + subcort
                            3: [3], # WM = WM 
                            0: [7, 5],
                           } 

def reduce_feta(fdir="/projects/datasets/FETA/FastSurfer", 
                idir="/projects/datasets/FETA/iBEAT/processing", 
                sbj="sub-001", mapping_fi=REDUCEMAPPING_FETA_IBEAT, mapping_f=REDUCEMAPPING_FETA):
    # FETA mapping
    # load image FETA
    gt = nib.load(opj(fdir, sbj, "dseg_orig.nii.gz"))
    gt_d = np.asanyarray(gt.dataobj)
    gt_new = deepcopy(gt_d)
    
    # Perform the mapping (FETA Gt to ibeat)
    for i in [2, 1, 3, 0]:
        mask = np.in1d(gt_d, mapping_fi[i]).reshape(gt_d.shape)
        # Reduce ground truth to labels we want
        gt_new[mask] = i
    
    save_image(gt_new, gt.affine, gt.header, save_as=opj(fdir, sbj, "dseg_ibeat.nii.gz"))
    
    # load images iBEAT
    pred_full = nib.load(opj(idir, sbj, "T2-skullstripped-tissue.nii.gz"))
    pred_d = np.asanyarray(pred_full.dataobj)
                         
    # load prediction (without cerebellum)
    pred_rmcere = nib.load(opj(idir, sbj, "T2-skullstripped-rmcere-tissue.nii.gz"))
    pred_d2 = np.asanyarray(pred_rmcere.dataobj)
    
    # Mask cerebellum WM/GM segmentation, remove parts from ibeat which can not be mapped correctly (brainstem)
    pred_d2[pred_d == 1] = 1
    pred_d2[(gt_d == 7)] = 0
    save_image(pred_d2, pred_rmcere.affine, pred_rmcere.header, save_as=opj(fdir, sbj, "aseg.TestingSet_ibeat_T2_05_T2_full.nii.gz"))
    

    # load images FastSurferVINN
    pred_vinn = nib.load(opj(fdir, sbj, "aseg.TestingSet_FastSurferVINN_T2_05_T2_LatentAug_full.mgz"))
    pred_v = np.asanyarray(pred_vinn.dataobj)
    pred_v_new = deepcopy(pred_v)
    
    
    # Perform the mapping (VINN to FETA)
    for i in [3, 2, 1, 5, 6, 7, 0]:
        mask = np.in1d(pred_v, mapping_f[i]).reshape(pred_v.shape)
        # Reduce prediction to labels we have
        pred_v_new[mask] = i
    save_image(pred_v_new, pred_vinn.affine, pred_vinn.header, save_as=opj(fdir, sbj, "aseg.TestingSet_VINN_T2_05_T2_LatentAug_full.nii.gz"))
    
    # Mask intra cranial in ground truth
    gt_d[np.in1d(pred_v, [88, 85]).reshape(pred_v.shape)] = 0
    gt_d[(gt_d == 4)] = 1
    save_image(gt_d, gt.affine, gt.header, save_as=opj(fdir, sbj, "dseg_vinn.nii.gz"))
    return gt_d, pred_d2, gt_new, pred_v_new

def vinn_to_feta(fdir, sbj, mapping_f=REDUCEMAPPING_FETA):
    # load images FastSurferVINN
    sbj_dir = opj(fdir, sbj)
    all_pred = glob.glob(sbj_dir + "/aseg.TestingSet_FastSurferVINN_T2*_full.mgz")
    for pred in all_pred:
        pred_vinn = nib.load(pred)
        pred_v = np.asanyarray(pred_vinn.dataobj)
        pred_v_new = np.zeros(pred_v.shape)

        # Perform the mapping (VINN to FETA)
        for i in [3, 2, 1, 4, 5, 6, 7, 0]:
            mask = np.in1d(pred_v, mapping_f[i]).reshape(pred_v.shape)
            # Reduce prediction to labels we have
            pred_v_new[mask] = i
        save_image_as_nifti(image_to_save=pred_v_new, affine=pred_vinn.affine, 
                            header=pred_vinn.header, 
                            save_as=pred[:-4] + "_mapped.mgz")
    
    # Mask intra cranial in ground truth
    # load image FETA
    gt = nib.load(opj(sbj_dir, "dseg_orig.nii.gz"))
    gt_d = np.asanyarray(gt.dataobj)
    
    gt_d[np.in1d(pred_v, [88, 85]).reshape(pred_v.shape)] = 0
    #gt_d[(gt_d == 4)] = 1
    save_image_as_nifti(image_to_save=gt_d, affine=gt.affine, 
                        header=gt.header, save_as=opj(sbj_dir, "dseg_vinn.mgz"))
    return gt_d, pred_v_new

# Hippocampus = WM and GM?
# Exclude 44, 45, 85, 88 where gt is not 6

In [38]:
import glob
fdir="/groups/ag-reuter/projects/datasets/FETA/FastSurfer"
feta = glob.glob(fdir + "/sub-*")
for sub in feta:
    sbj = sub.split("/")[-1]
    try:
        output = vinn_to_feta(fdir=fdir, sbj=sbj)
    except FileNotFoundError as e:
        print(f"Subject {sbj} failed: Error {e}")

Save image as:  /groups/ag-reuter/projects/datasets/FETA/FastSurfer/sub-001/aseg.TestingSet_FastSurferVINN_T2_05_T2_LatentAug_full_mapped.mgz
Save image as:  /groups/ag-reuter/projects/datasets/FETA/FastSurfer/sub-001/aseg.TestingSet_FastSurferVINN_T2_05_T2_LatentAugPlus_full_mapped.mgz
Save image as:  /groups/ag-reuter/projects/datasets/FETA/FastSurfer/sub-001/aseg.TestingSet_FastSurferVINN_T2_05_T2_LatentAugPlusSynthSegFull_full_mapped.mgz
Save image as:  /groups/ag-reuter/projects/datasets/FETA/FastSurfer/sub-001/aseg.TestingSet_FastSurferVINN_T2_05_T2_LatentAugPlusSynthSegMix_full_mapped.mgz
Save image as:  /groups/ag-reuter/projects/datasets/FETA/FastSurfer/sub-001/dseg_vinn.mgz
Save image as:  /groups/ag-reuter/projects/datasets/FETA/FastSurfer/sub-002/aseg.TestingSet_FastSurferVINN_T2_05_T2_LatentAug_full_mapped.mgz
Save image as:  /groups/ag-reuter/projects/datasets/FETA/FastSurfer/sub-002/aseg.TestingSet_FastSurferVINN_T2_05_T2_LatentAugPlus_full_mapped.mgz
Save image as:  /gr

Save image as:  /groups/ag-reuter/projects/datasets/FETA/FastSurfer/sub-012/dseg_vinn.mgz
Save image as:  /groups/ag-reuter/projects/datasets/FETA/FastSurfer/sub-013/aseg.TestingSet_FastSurferVINN_T2_05_T2_LatentAug_full_mapped.mgz
Save image as:  /groups/ag-reuter/projects/datasets/FETA/FastSurfer/sub-013/aseg.TestingSet_FastSurferVINN_T2_05_T2_LatentAugPlusSynthSegMix_full_mapped.mgz
Save image as:  /groups/ag-reuter/projects/datasets/FETA/FastSurfer/sub-013/aseg.TestingSet_FastSurferVINN_T2_05_T2_LatentAugPlus_full_mapped.mgz
Save image as:  /groups/ag-reuter/projects/datasets/FETA/FastSurfer/sub-013/aseg.TestingSet_FastSurferVINN_T2_05_T2_LatentAugPlusSynthSegFull_full_mapped.mgz
Save image as:  /groups/ag-reuter/projects/datasets/FETA/FastSurfer/sub-013/dseg_vinn.mgz
Save image as:  /groups/ag-reuter/projects/datasets/FETA/FastSurfer/sub-014/aseg.TestingSet_FastSurferVINN_T2_05_T2_LatentAug_full_mapped.mgz
Save image as:  /groups/ag-reuter/projects/datasets/FETA/FastSurfer/sub-014

Save image as:  /groups/ag-reuter/projects/datasets/FETA/FastSurfer/sub-024/dseg_vinn.mgz
Save image as:  /groups/ag-reuter/projects/datasets/FETA/FastSurfer/sub-025/aseg.TestingSet_FastSurferVINN_T2_05_T2_LatentAug_full_mapped.mgz
Save image as:  /groups/ag-reuter/projects/datasets/FETA/FastSurfer/sub-025/aseg.TestingSet_FastSurferVINN_T2_05_T2_LatentAugPlusSynthSegMix_full_mapped.mgz
Save image as:  /groups/ag-reuter/projects/datasets/FETA/FastSurfer/sub-025/aseg.TestingSet_FastSurferVINN_T2_05_T2_LatentAugPlus_full_mapped.mgz
Save image as:  /groups/ag-reuter/projects/datasets/FETA/FastSurfer/sub-025/aseg.TestingSet_FastSurferVINN_T2_05_T2_LatentAugPlusSynthSegFull_full_mapped.mgz
Save image as:  /groups/ag-reuter/projects/datasets/FETA/FastSurfer/sub-025/dseg_vinn.mgz
Save image as:  /groups/ag-reuter/projects/datasets/FETA/FastSurfer/sub-026/aseg.TestingSet_FastSurferVINN_T2_05_T2_LatentAug_full_mapped.mgz
Save image as:  /groups/ag-reuter/projects/datasets/FETA/FastSurfer/sub-026

Save image as:  /groups/ag-reuter/projects/datasets/FETA/FastSurfer/sub-036/dseg_vinn.mgz
Save image as:  /groups/ag-reuter/projects/datasets/FETA/FastSurfer/sub-037/aseg.TestingSet_FastSurferVINN_T2_05_T2_LatentAug_full_mapped.mgz
Save image as:  /groups/ag-reuter/projects/datasets/FETA/FastSurfer/sub-037/aseg.TestingSet_FastSurferVINN_T2_05_T2_LatentAugPlusSynthSegMix_full_mapped.mgz
Save image as:  /groups/ag-reuter/projects/datasets/FETA/FastSurfer/sub-037/aseg.TestingSet_FastSurferVINN_T2_05_T2_LatentAugPlus_full_mapped.mgz
Save image as:  /groups/ag-reuter/projects/datasets/FETA/FastSurfer/sub-037/aseg.TestingSet_FastSurferVINN_T2_05_T2_LatentAugPlusSynthSegFull_full_mapped.mgz
Save image as:  /groups/ag-reuter/projects/datasets/FETA/FastSurfer/sub-037/dseg_vinn.mgz
Save image as:  /groups/ag-reuter/projects/datasets/FETA/FastSurfer/sub-038/aseg.TestingSet_FastSurferVINN_T2_05_T2_LatentAug_full_mapped.mgz
Save image as:  /groups/ag-reuter/projects/datasets/FETA/FastSurfer/sub-038

Save image as:  /groups/ag-reuter/projects/datasets/FETA/FastSurfer/sub-048/dseg_vinn.mgz
Save image as:  /groups/ag-reuter/projects/datasets/FETA/FastSurfer/sub-049/aseg.TestingSet_FastSurferVINN_T2_05_T2_LatentAug_full_mapped.mgz
Save image as:  /groups/ag-reuter/projects/datasets/FETA/FastSurfer/sub-049/aseg.TestingSet_FastSurferVINN_T2_05_T2_LatentAugPlusSynthSegMix_full_mapped.mgz
Save image as:  /groups/ag-reuter/projects/datasets/FETA/FastSurfer/sub-049/aseg.TestingSet_FastSurferVINN_T2_05_T2_LatentAugPlus_full_mapped.mgz
Save image as:  /groups/ag-reuter/projects/datasets/FETA/FastSurfer/sub-049/aseg.TestingSet_FastSurferVINN_T2_05_T2_LatentAugPlusSynthSegFull_full_mapped.mgz
Save image as:  /groups/ag-reuter/projects/datasets/FETA/FastSurfer/sub-049/dseg_vinn.mgz
Save image as:  /groups/ag-reuter/projects/datasets/FETA/FastSurfer/sub-050/aseg.TestingSet_FastSurferVINN_T2_05_T2_LatentAug_full_mapped.mgz
Save image as:  /groups/ag-reuter/projects/datasets/FETA/FastSurfer/sub-050

...

Save image as:  /groups/ag-reuter/projects/datasets/FETA/FastSurfer/sub-072/dseg_vinn.mgz
Save image as:  /groups/ag-reuter/projects/datasets/FETA/FastSurfer/sub-073/aseg.TestingSet_FastSurferVINN_T2_05_T2_LatentAug_full_mapped.mgz
Save image as:  /groups/ag-reuter/projects/datasets/FETA/FastSurfer/sub-073/aseg.TestingSet_FastSurferVINN_T2_05_T2_LatentAugPlus_full_mapped.mgz
Save image as:  /groups/ag-reuter/projects/datasets/FETA/FastSurfer/sub-073/aseg.TestingSet_FastSurferVINN_T2_05_T2_LatentAugPlusSynthSegMix_full_mapped.mgz
Save image as:  /groups/ag-reuter/projects/datasets/FETA/FastSurfer/sub-073/aseg.TestingSet_FastSurferVINN_T2_05_T2_LatentAugPlusSynthSegFull_full_mapped.mgz
Save image as:  /groups/ag-reuter/projects/datasets/FETA/FastSurfer/sub-073/dseg_vinn.mgz
Save image as:  /groups/ag-reuter/projects/datasets/FETA/FastSurfer/sub-074/aseg.TestingSet_FastSurferVINN_T2_05_T2_LatentAug_full_mapped.mgz
Save image as:  /groups/ag-reuter/projects/datasets/FETA/FastSurfer/sub-074

Save image as:  /groups/ag-reuter/projects/datasets/FETA/FastSurfer/sub-104/dseg_vinn.mgz
Save image as:  /groups/ag-reuter/projects/datasets/FETA/FastSurfer/sub-105/aseg.TestingSet_FastSurferVINN_T2_05_T2_LatentAug_full_mapped.mgz
Save image as:  /groups/ag-reuter/projects/datasets/FETA/FastSurfer/sub-105/aseg.TestingSet_FastSurferVINN_T2_05_T2_LatentAugPlusSynthSegMix_full_mapped.mgz
Save image as:  /groups/ag-reuter/projects/datasets/FETA/FastSurfer/sub-105/aseg.TestingSet_FastSurferVINN_T2_05_T2_LatentAugPlus_full_mapped.mgz
Save image as:  /groups/ag-reuter/projects/datasets/FETA/FastSurfer/sub-105/aseg.TestingSet_FastSurferVINN_T2_05_T2_LatentAugPlusSynthSegFull_full_mapped.mgz
Save image as:  /groups/ag-reuter/projects/datasets/FETA/FastSurfer/sub-105/dseg_vinn.mgz
Save image as:  /groups/ag-reuter/projects/datasets/FETA/FastSurfer/sub-106/aseg.TestingSet_FastSurferVINN_T2_05_T2_LatentAug_full_mapped.mgz
Save image as:  /groups/ag-reuter/projects/datasets/FETA/FastSurfer/sub-106

Save image as:  /groups/ag-reuter/projects/datasets/FETA/FastSurfer/sub-116/dseg_vinn.mgz
Save image as:  /groups/ag-reuter/projects/datasets/FETA/FastSurfer/sub-117/aseg.TestingSet_FastSurferVINN_T2_05_T2_LatentAug_full_mapped.mgz
Save image as:  /groups/ag-reuter/projects/datasets/FETA/FastSurfer/sub-117/aseg.TestingSet_FastSurferVINN_T2_05_T2_LatentAugPlusSynthSegFull_full_mapped.mgz
Save image as:  /groups/ag-reuter/projects/datasets/FETA/FastSurfer/sub-117/aseg.TestingSet_FastSurferVINN_T2_05_T2_LatentAugPlus_full_mapped.mgz
Save image as:  /groups/ag-reuter/projects/datasets/FETA/FastSurfer/sub-117/aseg.TestingSet_FastSurferVINN_T2_05_T2_LatentAugPlusSynthSegMix_full_mapped.mgz
Save image as:  /groups/ag-reuter/projects/datasets/FETA/FastSurfer/sub-117/dseg_vinn.mgz
Save image as:  /groups/ag-reuter/projects/datasets/FETA/FastSurfer/sub-118/aseg.TestingSet_FastSurferVINN_T2_05_T2_LatentAug_full_mapped.mgz
Save image as:  /groups/ag-reuter/projects/datasets/FETA/FastSurfer/sub-118

Save image as:  /groups/ag-reuter/projects/datasets/FETA/FastSurfer/sub-128/dseg_vinn.mgz
Save image as:  /groups/ag-reuter/projects/datasets/FETA/FastSurfer/sub-129/aseg.TestingSet_FastSurferVINN_T2_05_T2_LatentAug_full_mapped.mgz
Save image as:  /groups/ag-reuter/projects/datasets/FETA/FastSurfer/sub-129/aseg.TestingSet_FastSurferVINN_T2_05_T2_LatentAugPlusSynthSegFull_full_mapped.mgz
Save image as:  /groups/ag-reuter/projects/datasets/FETA/FastSurfer/sub-129/aseg.TestingSet_FastSurferVINN_T2_05_T2_LatentAugPlusSynthSegMix_full_mapped.mgz
Save image as:  /groups/ag-reuter/projects/datasets/FETA/FastSurfer/sub-129/aseg.TestingSet_FastSurferVINN_T2_05_T2_LatentAugPlus_full_mapped.mgz
Save image as:  /groups/ag-reuter/projects/datasets/FETA/FastSurfer/sub-129/dseg_vinn.mgz
Save image as:  /groups/ag-reuter/projects/datasets/FETA/FastSurfer/sub-130/aseg.TestingSet_FastSurferVINN_T2_05_T2_LatentAug_full_mapped.mgz
Save image as:  /groups/ag-reuter/projects/datasets/FETA/FastSurfer/sub-130

Save image as:  /groups/ag-reuter/projects/datasets/FETA/FastSurfer/sub-140/dseg_vinn.mgz


In [81]:
gt_new_vinn, pred_new_ibeat, gt_new_ibeat, pred_new_vinn = reduce_feta()

saving image as /projects/datasets/FETA/FastSurfer/sub-001/aseg.TestingSet_ibeat_T2_05_T2_full.nii.gz
saving image as /projects/datasets/FETA/FastSurfer/sub-001/dseg_ibeat.nii.gz
saving image as /projects/datasets/FETA/FastSurfer/sub-001/aseg.TestingSet_VINN_T2_05_T2_LatentAug_full.nii.gz
saving image as /projects/datasets/FETA/FastSurfer/sub-001/dseg_vinn.nii.gz


In [85]:
import glob
feta = glob.glob("/projects/datasets/FETA/FastSurfer/sub-*")
for sub in feta:
    sbj = sub.split("/")[-1]
    try:
        output = reduce_feta(sbj=sbj)
    except FileNotFoundError as e:
        print(f"Subject {sbj} failed: Error {e}")

saving image as /projects/datasets/FETA/FastSurfer/sub-001/aseg.TestingSet_ibeat_T2_05_T2_full.nii.gz
saving image as /projects/datasets/FETA/FastSurfer/sub-001/dseg_ibeat.nii.gz
saving image as /projects/datasets/FETA/FastSurfer/sub-001/aseg.TestingSet_VINN_T2_05_T2_LatentAug_full.nii.gz
saving image as /projects/datasets/FETA/FastSurfer/sub-001/dseg_vinn.nii.gz
saving image as /projects/datasets/FETA/FastSurfer/sub-002/aseg.TestingSet_ibeat_T2_05_T2_full.nii.gz
saving image as /projects/datasets/FETA/FastSurfer/sub-002/dseg_ibeat.nii.gz
saving image as /projects/datasets/FETA/FastSurfer/sub-002/aseg.TestingSet_VINN_T2_05_T2_LatentAug_full.nii.gz
saving image as /projects/datasets/FETA/FastSurfer/sub-002/dseg_vinn.nii.gz
saving image as /projects/datasets/FETA/FastSurfer/sub-003/aseg.TestingSet_ibeat_T2_05_T2_full.nii.gz
saving image as /projects/datasets/FETA/FastSurfer/sub-003/dseg_ibeat.nii.gz
saving image as /projects/datasets/FETA/FastSurfer/sub-003/aseg.TestingSet_VINN_T2_05_T2_

saving image as /projects/datasets/FETA/FastSurfer/sub-024/aseg.TestingSet_ibeat_T2_05_T2_full.nii.gz
saving image as /projects/datasets/FETA/FastSurfer/sub-024/dseg_ibeat.nii.gz
saving image as /projects/datasets/FETA/FastSurfer/sub-024/aseg.TestingSet_VINN_T2_05_T2_LatentAug_full.nii.gz
saving image as /projects/datasets/FETA/FastSurfer/sub-024/dseg_vinn.nii.gz
saving image as /projects/datasets/FETA/FastSurfer/sub-025/aseg.TestingSet_ibeat_T2_05_T2_full.nii.gz
saving image as /projects/datasets/FETA/FastSurfer/sub-025/dseg_ibeat.nii.gz
saving image as /projects/datasets/FETA/FastSurfer/sub-025/aseg.TestingSet_VINN_T2_05_T2_LatentAug_full.nii.gz
saving image as /projects/datasets/FETA/FastSurfer/sub-025/dseg_vinn.nii.gz
saving image as /projects/datasets/FETA/FastSurfer/sub-026/aseg.TestingSet_ibeat_T2_05_T2_full.nii.gz
saving image as /projects/datasets/FETA/FastSurfer/sub-026/dseg_ibeat.nii.gz
saving image as /projects/datasets/FETA/FastSurfer/sub-026/aseg.TestingSet_VINN_T2_05_T2_

saving image as /projects/datasets/FETA/FastSurfer/sub-046/aseg.TestingSet_VINN_T2_05_T2_LatentAug_full.nii.gz
saving image as /projects/datasets/FETA/FastSurfer/sub-046/dseg_vinn.nii.gz
saving image as /projects/datasets/FETA/FastSurfer/sub-047/aseg.TestingSet_ibeat_T2_05_T2_full.nii.gz
saving image as /projects/datasets/FETA/FastSurfer/sub-047/dseg_ibeat.nii.gz
saving image as /projects/datasets/FETA/FastSurfer/sub-047/aseg.TestingSet_VINN_T2_05_T2_LatentAug_full.nii.gz
saving image as /projects/datasets/FETA/FastSurfer/sub-047/dseg_vinn.nii.gz
saving image as /projects/datasets/FETA/FastSurfer/sub-048/aseg.TestingSet_ibeat_T2_05_T2_full.nii.gz
saving image as /projects/datasets/FETA/FastSurfer/sub-048/dseg_ibeat.nii.gz
saving image as /projects/datasets/FETA/FastSurfer/sub-048/aseg.TestingSet_VINN_T2_05_T2_LatentAug_full.nii.gz
saving image as /projects/datasets/FETA/FastSurfer/sub-048/dseg_vinn.nii.gz
saving image as /projects/datasets/FETA/FastSurfer/sub-049/aseg.TestingSet_ibeat_

saving image as /projects/datasets/FETA/FastSurfer/sub-069/aseg.TestingSet_ibeat_T2_05_T2_full.nii.gz
saving image as /projects/datasets/FETA/FastSurfer/sub-069/dseg_ibeat.nii.gz
saving image as /projects/datasets/FETA/FastSurfer/sub-069/aseg.TestingSet_VINN_T2_05_T2_LatentAug_full.nii.gz
saving image as /projects/datasets/FETA/FastSurfer/sub-069/dseg_vinn.nii.gz
saving image as /projects/datasets/FETA/FastSurfer/sub-070/aseg.TestingSet_ibeat_T2_05_T2_full.nii.gz
saving image as /projects/datasets/FETA/FastSurfer/sub-070/dseg_ibeat.nii.gz
saving image as /projects/datasets/FETA/FastSurfer/sub-070/aseg.TestingSet_VINN_T2_05_T2_LatentAug_full.nii.gz
saving image as /projects/datasets/FETA/FastSurfer/sub-070/dseg_vinn.nii.gz
saving image as /projects/datasets/FETA/FastSurfer/sub-071/aseg.TestingSet_ibeat_T2_05_T2_full.nii.gz
saving image as /projects/datasets/FETA/FastSurfer/sub-071/dseg_ibeat.nii.gz
saving image as /projects/datasets/FETA/FastSurfer/sub-071/aseg.TestingSet_VINN_T2_05_T2_

saving image as /projects/datasets/FETA/FastSurfer/sub-111/aseg.TestingSet_VINN_T2_05_T2_LatentAug_full.nii.gz
saving image as /projects/datasets/FETA/FastSurfer/sub-111/dseg_vinn.nii.gz
saving image as /projects/datasets/FETA/FastSurfer/sub-112/aseg.TestingSet_ibeat_T2_05_T2_full.nii.gz
saving image as /projects/datasets/FETA/FastSurfer/sub-112/dseg_ibeat.nii.gz
saving image as /projects/datasets/FETA/FastSurfer/sub-112/aseg.TestingSet_VINN_T2_05_T2_LatentAug_full.nii.gz
saving image as /projects/datasets/FETA/FastSurfer/sub-112/dseg_vinn.nii.gz
saving image as /projects/datasets/FETA/FastSurfer/sub-113/aseg.TestingSet_ibeat_T2_05_T2_full.nii.gz
saving image as /projects/datasets/FETA/FastSurfer/sub-113/dseg_ibeat.nii.gz
saving image as /projects/datasets/FETA/FastSurfer/sub-113/aseg.TestingSet_VINN_T2_05_T2_LatentAug_full.nii.gz
saving image as /projects/datasets/FETA/FastSurfer/sub-113/dseg_vinn.nii.gz
saving image as /projects/datasets/FETA/FastSurfer/sub-114/aseg.TestingSet_ibeat_

saving image as /projects/datasets/FETA/FastSurfer/sub-138/aseg.TestingSet_ibeat_T2_05_T2_full.nii.gz
Subject sub-CC01103XX06_ses-101930 failed: Error No such file or no access: '/projects/datasets/FETA/FastSurfer/sub-138/aseg.TestingSet_FastSurferVINN_T2_05_T2_LatentAug_full.mgz'
saving image as /projects/datasets/FETA/FastSurfer/sub-139/aseg.TestingSet_ibeat_T2_05_T2_full.nii.gz
Subject sub-CC01103XX06_ses-101930 failed: Error No such file or no access: '/projects/datasets/FETA/FastSurfer/sub-139/aseg.TestingSet_FastSurferVINN_T2_05_T2_LatentAug_full.mgz'
saving image as /projects/datasets/FETA/FastSurfer/sub-140/aseg.TestingSet_ibeat_T2_05_T2_full.nii.gz
Subject sub-CC01103XX06_ses-101930 failed: Error No such file or no access: '/projects/datasets/FETA/FastSurfer/sub-140/aseg.TestingSet_FastSurferVINN_T2_05_T2_LatentAug_full.mgz'


In [12]:
# "sub-CC00600XX06_ses-175800"
# CSF (1) = Left-/Right-Lateral-Ventricle, CSF
# GM (2) = Hippocampus, Amygdala, GM Parts
# Exclude (4): Brainstem, intra-cranial BG, Cerebellum
# WM (3): CC, WM parts + Thalamus, 
REDUCEDMAPPING_IBEAT = {1: [83, 49, 50],
           2: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 
               20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
               35, 36, 37, 38, 39], #40, 41, 42, 43, 44, 45, 46, 47,
           3: [48, 51, 52, 53, 54, 55, 56, 57, 
               58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 
               68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 
               78, 79, 80, 81, 82],
           0: [17, 18, 19, 40, 41, 42, 43, 44, 45, 46, 47, 84, 85, 86, 87, 88]}

base="/groups/ag-reuter"

def reduce_ibeat(idir= base + "/projects/datasets/dHCP/iBEAT/processing", fdir=base + "/projects/datasets/dHCP/Data", 
                 sbj="sub-CC00408XX12_ses-125500", mapping=REDUCEDMAPPING_IBEAT):
    # iBEAT mapping
    # load image
    gt = nib.load(opj(fdir, sbj, sbj + "_desc-drawem88_dseg_min.nii.gz"))
    gt_d = np.asanyarray(gt.dataobj)
    gt_new = deepcopy(gt_d)

    # load prediction (full)
    pred_full = nib.load(opj(idir, sbj, "T2-skullstripped-tissue.nii.gz"))
    pred_d = np.asanyarray(pred_full.dataobj)
                         
    # load prediction (without cerebellum)
    pred_rmcere = nib.load(opj(idir, sbj, "T2-skullstripped-rmcere-tissue.nii.gz"))
    pred_d2 = np.asanyarray(pred_rmcere.dataobj)
    
    # Mask cerebellum WM/GM segmentation, remove parts from ibeat which can not be mapped correctly (under)
    pred_d2[pred_d == 1] = 1
    pred_d2[(pred_d == 2) & (pred_d2 == 0)] = 5
    pred_d2[(pred_d == 3) & (pred_d2 == 0)] = 5
    pred_d2[np.in1d(gt_d, mapping[0][2:]).reshape(gt_d.shape)] = 5
    pred_d2[np.in1d(gt_d, [84]).reshape(gt_d.shape)] = 0
    
    # Perform the mapping
    for i in [2, 1, 3, 0]:
        mask = np.in1d(gt_d, mapping[i]).reshape(gt_d.shape)
        # Reduce ground truth to labels we want
        gt_new[mask] = i
    
    #save_image(gt_new, gt.affine, gt.header, save_as=opj(fdir, sbj, sbj + "_ibeat_min.nii.gz"))
    save_image(pred_d2, pred_full.affine, pred_full.header, 
               save_as=opj(fdir, sbj, "mri", "aseg.TestingSet_ibeat_T2_05_T2_full_v3.nii.gz"))
    return gt, gt_new, pred_full, pred_d2

  
                           
                           

In [13]:
gt_head, gt_fin, pred_head, pred_fin = reduce_ibeat()

saving image as /groups/ag-reuter/projects/datasets/dHCP/Data/sub-CC00408XX12_ses-125500/mri/aseg.TestingSet_ibeat_T2_05_T2_full_v3.nii.gz


In [8]:
def save_image(image_to_save, affine, header, save_as, dtype_set=np.int16):
    """
        Function to save a given file as a nifti-image
        :param ndarray image_to_save: image with dimensions (height, width, depth) to be saved in nifti format
        :param ndarray affine: affine information
        :param ndarray header: header information
        :param str save_as: name and directory where the nifti should be stored
        .param dtype dtype_set: dtype under which image should be saved (int by default)
        :return: void
        """
    print(f"saving image as {save_as}")
    header.set_data_dtype(dtype_set)
    nifti_new = nib.nifti1.Nifti1Pair(image_to_save, affine, header)
    nifti_new.set_data_dtype(np.dtype(dtype_set))  # not uint8 if aparc!!! (only goes till 255)
    nib.nifti1.save(nifti_new, save_as)

In [8]:
training_samples = pd.read_csv(training, sep="\t")

In [9]:
for samples in training_samples["SubjectFix"]:
    try:
        output = reduce_ibeat(sbj=samples)
    except FileNotFoundError as e:
        print(f"Subject {samples} failed: Error {e}")

saving image as /projects/datasets/dHCP/Data/sub-CC00284AN13_ses-90701/sub-CC00284AN13_ses-90701_ibeat_min.nii.gz
saving image as /projects/datasets/dHCP/Data/sub-CC00284AN13_ses-90701/mri/aseg.TestingSet_ibeat_T2_05_T2_full.nii.gz
saving image as /projects/datasets/dHCP/Data/sub-CC00829XX21_ses-17610/sub-CC00829XX21_ses-17610_ibeat_min.nii.gz
saving image as /projects/datasets/dHCP/Data/sub-CC00829XX21_ses-17610/mri/aseg.TestingSet_ibeat_T2_05_T2_full.nii.gz
saving image as /projects/datasets/dHCP/Data/sub-CC00136BN13_ses-45000/sub-CC00136BN13_ses-45000_ibeat_min.nii.gz
saving image as /projects/datasets/dHCP/Data/sub-CC00136BN13_ses-45000/mri/aseg.TestingSet_ibeat_T2_05_T2_full.nii.gz
saving image as /projects/datasets/dHCP/Data/sub-CC01234AN14_ses-155030/sub-CC01234AN14_ses-155030_ibeat_min.nii.gz
saving image as /projects/datasets/dHCP/Data/sub-CC01234AN14_ses-155030/mri/aseg.TestingSet_ibeat_T2_05_T2_full.nii.gz
saving image as /projects/datasets/dHCP/Data/sub-CC00136AN13_ses-4510

saving image as /projects/datasets/dHCP/Data/sub-CC00206XX08_ses-70100/mri/aseg.TestingSet_ibeat_T2_05_T2_full.nii.gz
saving image as /projects/datasets/dHCP/Data/sub-CC00483XX14_ses-144200/sub-CC00483XX14_ses-144200_ibeat_min.nii.gz
saving image as /projects/datasets/dHCP/Data/sub-CC00483XX14_ses-144200/mri/aseg.TestingSet_ibeat_T2_05_T2_full.nii.gz
saving image as /projects/datasets/dHCP/Data/sub-CC00441XX13_ses-132902/sub-CC00441XX13_ses-132902_ibeat_min.nii.gz
saving image as /projects/datasets/dHCP/Data/sub-CC00441XX13_ses-132902/mri/aseg.TestingSet_ibeat_T2_05_T2_full.nii.gz
saving image as /projects/datasets/dHCP/Data/sub-CC00304XX07_ses-111600/sub-CC00304XX07_ses-111600_ibeat_min.nii.gz
saving image as /projects/datasets/dHCP/Data/sub-CC00304XX07_ses-111600/mri/aseg.TestingSet_ibeat_T2_05_T2_full.nii.gz
saving image as /projects/datasets/dHCP/Data/sub-CC01004XX06_ses-51630/sub-CC01004XX06_ses-51630_ibeat_min.nii.gz
saving image as /projects/datasets/dHCP/Data/sub-CC01004XX06_se

saving image as /projects/datasets/dHCP/Data/sub-CC00095XX14_ses-33501/sub-CC00095XX14_ses-33501_ibeat_min.nii.gz
saving image as /projects/datasets/dHCP/Data/sub-CC00095XX14_ses-33501/mri/aseg.TestingSet_ibeat_T2_05_T2_full.nii.gz
saving image as /projects/datasets/dHCP/Data/sub-CC00512XX09_ses-150700/sub-CC00512XX09_ses-150700_ibeat_min.nii.gz
saving image as /projects/datasets/dHCP/Data/sub-CC00512XX09_ses-150700/mri/aseg.TestingSet_ibeat_T2_05_T2_full.nii.gz
saving image as /projects/datasets/dHCP/Data/sub-CC00473XX12_ses-140100/sub-CC00473XX12_ses-140100_ibeat_min.nii.gz
saving image as /projects/datasets/dHCP/Data/sub-CC00473XX12_ses-140100/mri/aseg.TestingSet_ibeat_T2_05_T2_full.nii.gz
saving image as /projects/datasets/dHCP/Data/sub-CC00076XX11_ses-28700/sub-CC00076XX11_ses-28700_ibeat_min.nii.gz
saving image as /projects/datasets/dHCP/Data/sub-CC00076XX11_ses-28700/mri/aseg.TestingSet_ibeat_T2_05_T2_full.nii.gz
saving image as /projects/datasets/dHCP/Data/sub-CC00649XX23_ses-1

In [55]:
import sys
sys.path.append(base + "master-theses/henschell")
sys.path.append(base + "master-theses/henschell/SuperResSurfer")

from SuperResSurfer.SuperResSegm.utils.metrics import evaluate_metrics

In [63]:
result = evaluate_metrics(gt_fin, pred_fin, [1, 2, 3])

In [64]:
result

{'dice': array([0.74932848, 0.7580927 , 0.87178041]),
 'surfaceAverageHausdorff': array([0.91588205, 0.85160651, 1.07245862])}

In [59]:
sbj="sub-CC00600XX06_ses-175800"
pred_name = opj(fdir, sbj, sbj + "_ibeat_min.nii.gz")
gt_name = opj(fdir, sbj, "mri", "aseg.TestingSet_ibeat_T2_05_T2_full.nii.gz")
pred_data = np.asanyarray(nib.load(pred_name).dataobj)
gt_data = np.asanyarray(nib.load(gt_name).dataobj)
result2 = evaluate_metrics(gt_data, pred_data, [1, 2, 3])

In [60]:
result2

{'dice': array([0.74932848, 0.7580927 , 0.87178041]),
 'surfaceAverageHausdorff': array([0.91588205, 0.85160651, 1.07245862])}